In [1]:
using Pkg
using Revise
Pkg.activate(".")

# add projects
Pkg.develop(url = "https://github.com/jcsyme/IterativeHeaps.jl")
Pkg.develop(url = "https://github.com/jcsyme/GraphDistanceAlgorithms.jl")
Pkg.develop(url = "https://github.com/jcsyme/GraphFragments.jl")
#Pkg.develop(url = "https://github.com/jcsyme/DiscreteGraphAlgorithms.jl")
Pkg.develop(path = "/Users/usuario/git/DiscreteGraphAlgorithms.jl")
Pkg.add("BenchmarkTools")


  Activating project at `~/git/RunDGA`
     Cloning git-repo `https://github.com/jcsyme/IterativeHeaps.jl`
Path `/Users/usuario/.julia/dev/IterativeHeaps` exists and looks like the correct repo. Using existing path.
   Resolving package versions...
  No Changes to `~/git/RunDGA/Project.toml`
  No Changes to `~/git/RunDGA/Manifest.toml`
     Cloning git-repo `https://github.com/jcsyme/GraphDistanceAlgorithms.jl`
Path `/Users/usuario/.julia/dev/GraphDistanceAlgorithms` exists and looks like the correct repo. Using existing path.
   Resolving package versions...
  No Changes to `~/git/RunDGA/Project.toml`
  No Changes to `~/git/RunDGA/Manifest.toml`
     Cloning git-repo `https://github.com/jcsyme/GraphFragments.jl`
Path `/Users/usuario/.julia/dev/GraphFragments` exists and looks like the correct repo. Using existing path.
   Resolving package versions...
  No Changes to `~/git/RunDGA/Project.toml`
  No Changes to `~/git/RunDGA/Manifest.toml`
   Resolving package versions...
  No Changes 

# Load Benchmarking and `Distributed`
- `Distributed` package is required to take advantage of parallelization
    - The `@everywhere` macro will be used below to ensure that parallelization can function using methods in each of the pacakges we need
    - Use `Sys.CPU_THREADS` to see how many cores (physical and virtual) you have at your disposal
- `BenchmarkTools` is used to compare performance

In [2]:
Sys.CPU_THREADS

10

In [3]:
using BenchmarkTools 
using Distributed

# add processes? the number of processes should be based on your system. You can use 
(nprocs() == 1) && addprocs(Sys.CPU_THREADS)

# load using everywhere macro to make sure cores can recognize objects
@everywhere using DiscreteGraphAlgorithms
@everywhere using Graphs
@everywhere using GraphDistanceAlgorithms
@everywhere using GraphFragments

In [4]:
?GraphFragments.fragmentation

Calculate the fragmentation of a graph (KPP-Negative)

# Constructs

```
fragmentation(
    graph::Union{AbstractGraph, Nothing}, 
    dict_arrays::Union{Dict{Symbol, Vector}, Dict{Symbol, DArray}, Nothing} = nothing;
    D_invs::Union{Matrix{Int64}, Matrix{Float64}, Nothing} = nothing,
    distance_algorithm::Symbol = :auto,
    parallel_approach::Symbol = :auto,
    use_distance_weighting::Bool = true,
    kwargs...
)
```

```
fragmentation(
    adj::Union{SparseMatrixCSC{Float64, Int64}, Matrix{Float64}};;
    kwargs...
)
```

## Function Arguments

  * `graph`: graph on which to calculate fragmentation
  * `dict_arrays`: optional dictionary mapping relevant algorithm keys to arrays–   DArrays, SharedArrays (not recommended unless size is very large), or    Vectors–storing intermediate calculations.

      * Only effective if fixing `distance_algorithm` to align with the arrays   that are passed
      * See ?GraphDistanceAlgorithms.spawn_arrays for more information on the    inputs required.

## Keyword Arguments

  * `D_invs`: optional matrix (with 0 diagonal) of inverse distances. Passing    this optional argument can speed up calculations considerably.

    **CAUTION** This function assumes that `D_invs` is complete with 0s along        the diagonal.
  * `distance_algorithm`: distance489503algorithm to use in computing distances. Called   if `D_invs` is not specified
  * `parallel_approach`: `fragmentation` will automatically try to implement    parallel calculation if `try_parallel(graph) == true`. `parallel_approach`   can take one of three values:

      * `:auto`: choose based on `try_parallel(graph)`
      * `:parallel`: Force a parallel implementation (slower on small graphs)
      * `:serial`: Force a serial implementation (slower on large graphs)
  * `use_distance_weighting`: use distance-weigthed fragmentation? If False,    defaults to adjacency only.
  * `kwargs...`: passed to distance algorithm. Include options for heap vectors    etc.


# Let's try calculating fragmentation on some graphs


In [5]:
# generate a random graph and a wheel graph
graph = Graphs.SimpleGraphs.random_regular_graph(1000, 5);
graph_wheel = Graphs.SimpleGraphs.wheel_graph(1000);


# Compare serial and distributed approach
- Simple functions that iterate over each source vertex and calculate a metric


###   Serial approach requires us to specify `parallel_approach = :serial`
Run once to compile before benchmarking

In [5]:
@time GraphFragments.fragmentation(graph, parallel_approach = :serial)

  0.506301 seconds (3.08 M allocations: 227.888 MiB, 8.72% gc time, 93.40% compilation time)


0.7734361885695218

In [49]:
@benchmark GraphFragments.fragmentation(graph, parallel_approach = :serial)

BenchmarkTools.Trial: 204 samples with 1 evaluation per sample.
 Range (min … max):  23.038 ms … 38.967 ms  ┊ GC (min … max): 0.00% … 39.78%
 Time  (median):     23.609 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.574 ms ±  3.329 ms  ┊ GC (mean ± σ):  3.68% ±  9.11%

   ██▃                                                         
  ▆███▇▆▆▁▆▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▆▄▄▄▆▆▁▄▁▁▁▁▄ ▆
  23 ms        Histogram: log(frequency) by time      38.7 ms <

 Memory estimate: 20.39 MiB, allocs estimate: 16537.

###   To take advantage of parallelization, we should be explicit about the algorithm we're using and leverage cache arrays

- It's better to be explicit and set `parallel_approach = :parallel`
- Use `spawn_arrays` to prealocate array space for a graph `graph`
    - allows for `DistributedArrays` (recommended) and `SharedArrays` (not-recommended, performs worse on most graphs)
- This applies to fragmentation calculation and use of `DiscreteGraphAlgorithms` and `IterativeHeaps`
- Call `spawn_arrays` to build a dictionary of pre-allocated, shared (accessible by different processes) arrays
    - Default are `type = :DistributedArrays` (`DArrays`), which are used for parallel runs
    - Note that, if running in serial, use `type = :Vector`
    - If you generate spawn arrays, make sure you pass the proper `parallel_approach` to the function as well
- Compile before benchmarking by running once


In [24]:
algo = :dijkstra_kary
dict_arrays = spawn_arrays(graph, algo; )

@time GraphFragments.fragmentation(
    graph, 
    dict_arrays; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

  0.021955 seconds (6.21 k allocations: 1.937 MiB)


0.7734773154106485

**NOTE:** `dict_arrays` is shared and used to store intermediate calculations, including heap variables, parents, and distances. Its contents should not be modified.

In [51]:
@benchmark GraphFragments.fragmentation(
    graph, 
    dict_arrays; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

BenchmarkTools.Trial: 554 samples with 1 evaluation per sample.
 Range (min … max):  7.959 ms … 23.865 ms  ┊ GC (min … max): 0.00% … 65.25%
 Time  (median):     8.485 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.028 ms ±  1.443 ms  ┊ GC (mean ± σ):  0.89% ±  4.72%

   ▂█▆                                                        
  ▄███▇▄▄▄▄▄▃▄▄▃▅▅▄▄▄▅▄▄▃▃▄▃▂▂▃▃▂▃▂▁▁▁▂▂▂▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▂▁▃ ▃
  7.96 ms        Histogram: frequency by time        13.3 ms <

 Memory estimate: 1.94 MiB, allocs estimate: 6226.

# Next, let's compare fragmentation calculation on the wheel graph

- Some graphs are relatively efficient in serial
- 1000 node wheel graph is only _slightly_ faster in parallel

In [52]:
# serial
@time GraphFragments.fragmentation(
    graph_wheel,
    parallel_approach = :serial
)

  0.011792 seconds (16.54 k allocations: 20.390 MiB)


0.498

In [13]:
@benchmark GraphFragments.fragmentation(
    graph_wheel, 
    parallel_approach = :serial
)

BenchmarkTools.Trial: 600 samples with 1 evaluation per sample.
 Range (min … max):  6.491 ms … 22.349 ms  ┊ GC (min … max):  0.00% … 64.96%
 Time  (median):     7.551 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.332 ms ±  1.681 ms  ┊ GC (mean ± σ):  11.60% ± 14.05%

     ▂▆▇█▃▁                                                   
  ▃▃▅██████▆▆▅▅▅▄▃▄▃▄▄▄▄▃▄▃▄▄▄▄▄▄▅▄▄▄▂▃▄▃▃▄▂▂▃▃▃▁▂▁▂▂▂▁▁▂▁▁▂ ▃
  6.49 ms        Histogram: frequency by time        13.3 ms <

 Memory estimate: 20.39 MiB, allocs estimate: 16537.

In [14]:
algo = :dijkstra_kary

dict_arrays_wheel = spawn_arrays(graph_wheel, algo; )

@time GraphFragments.fragmentation(
    graph_wheel, 
    dict_arrays_wheel; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

  0.008438 seconds (6.17 k allocations: 1.864 MiB)


0.498

In [15]:
@benchmark GraphFragments.fragmentation(
    graph_wheel, 
    dict_arrays_wheel; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

BenchmarkTools.Trial: 782 samples with 1 evaluation per sample.
 Range (min … max):  5.362 ms … 15.571 ms  ┊ GC (min … max): 0.00% … 60.58%
 Time  (median):     5.618 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.392 ms ±  1.314 ms  ┊ GC (mean ± σ):  1.35% ±  5.67%

  ▃█▇▃▂            ▃▃▃▂▁                                      
  █████▇▇▆▇▆▇▄▇▇▇▇▇██████▆█▇▇▆███▆▇▇▅▄▄▇▆▆▆▄▄▄▄▄▅▁▄▁▄▄▅▄▄▄▄▄ ▇
  5.36 ms      Histogram: log(frequency) by time     10.9 ms <

 Memory estimate: 1.86 MiB, allocs estimate: 6166.

# We can try different algorithms

One key thing users should note is that not all algorithms perform the same on similar graphs. For example:

- **Gradient Descent** (`graddesc_iterand`) can perform very poorly on larger graphs (large $|V|$) since it's an exhaustive algorithm, not greedy
- The **Genetic** and **Ant Colony Optimization** algorithms respond extensively to tuning, and can perform better on larger graphs, though users shuold be cognizant of stopping conditions.
    - For example, increasing the `population_size` will slow down the iteration speed, but can lead to a larger search space and fewer iterations
    - Additionally, the `num_elite` parameter may perform better when a smaller fraction of larger populations to encourage larger search spaces
    - Setting the `max_iter_no_improvement` can avoid repetitive searching that doesn't appear to improve 


Some key parameters that users should think about:

- `max_iteration`: maximum number of iterations for an algorithm to complete. This should be set for different algorithms, as iterations in--for example--the **Greedy Stochastic** (`gs_iterand`), **Gradient Descent** (`graddesc_iterand`), and **Simulated Annealing** (`sann_iterand`) algorithms run much more quickly than the iterations in the **Genetic** (`genetic_iterand`) and **Ant Colony Optimization** (`aco_iterand`) algorithms.
- `max_iter_no_improvement`: Setting this can prevent algorithms from running indefinitely if there is a continuous set of runs with no improvement


##  All algorithms call an `OptimizationParameters` object

- Used to store parameters for the algorithms
    - Different algorithms can share this object
- Note that the `OptimizationParameters` object requires a `GraphWrapper`
    - convert a graph using `graph_to_graph_wrapper`
    - See `?GraphWrapper` for more information--you can add information to improve the analytical experience, such as:
        - Add edge weights to the graph wrapper
        - Add vertex names to the graph wrapper
- Algorithm-specific options can be passed in a dictionary using the `opts` object
    - Algorithm-specific options contain an algorithmic prefix; e.g., the `population_size` parameter in `aco_iterand` (Ant Colony Optimization) is passed using `:aco_population_size` (the `aco_` prefix is prepended to `population_size`)
- We can pass a starting set--best guess--using the `S` key word in `OptimizationParameters`
    - In the example below--often a good starting point for fragmentation--we choose nodes with the highest betweenness centrality
        - Can choose other centrality measures as well--see `?get_default_kpp_nodes` for more inf ormation
- Note: the `OptimizationParameters` object can be used to specify the `parallel_approach` as one of the following:
    - `:auto` (choose based on graph size)
    - `:parallel` (used available processes, shown with `nprocs()`
    - `:serial` (use serial processes, better on small graphs)
    - **IMPORTANT** If you specify this approach **and** specify `dict_arrays_distance`, make sure you spawn the correct type. In `spawn_arrays`, use `type = :DistributedArray` (default) when running in parallel and `type = :Vector` when running serially. If you use `parallel_approach = :auto` (Default in `OptimizationParameters`, then the arrays are spawned a part of the `OptimizationParameters` object, and you don't have to worry about it.

In [26]:
n_remove = 3

algo = :dijkstra_kary
dict_arrays = spawn_arrays(graph, algo; )
graph_wrapped = graph_to_graph_wrapper(graph);
S0 = get_default_kpp_nodes(graph, n_remove); 

# first, we setup some parameters
params_graph_gs = OptimizationParameters(
    n_remove,
    graph_wrapped;
    dict_arrays_distance = dict_arrays,
    distance_algorithm = algo,
    max_iter = 2000,
    max_iter_no_improvement = 500,
    S = S0,
);

##  Try the Greedy Stochastic algorithm (`gs_iterand`)


In [15]:
@time DiscreteGraphAlgorithms.iterate(
    gs_iterand,
    params_graph_gs;
    log_interval = 100,
)

[ Info: 1 iterations complete with value 0.774237677653346
[ Info: 101 iterations complete with value 0.774237677653346
[ Info: 201 iterations complete with value 0.774237677653346
[ Info: 301 iterations complete with value 0.774237677653346
[ Info: 401 iterations complete with value 0.774237677653346
[ Info: 501 iterations complete with value 0.774237677653346
[ Info: Stochastic gradiest descent complete in 501 iterations.


  6.143070 seconds (5.71 M allocations: 1.169 GiB, 5.26% gc time, 13.44% compilation time)


(0.774237677653346, [239, 409, 586], SimpleGraph{Int64}(2485, [[39, 121, 855, 912, 997], [173, 342, 672, 758, 826], [108, 161, 637, 667, 876], [49, 319, 369, 487, 591], [209, 286, 517, 593, 929], [302, 432, 487, 638, 713], [151, 188, 596, 683, 773], [183, 359, 543, 665, 982], [72, 608, 746, 853, 884], [134, 320, 418, 861, 955]  …  [50, 104, 136, 332, 461], [349, 367, 591, 759, 945], [218, 370, 510, 894, 977], [281, 379, 434, 487, 721], [15, 172, 533, 717, 804], [80, 330, 528, 549, 898], [178, 198, 341, 366, 811], [67, 123, 191, 588, 730], [58, 337, 634, 773, 983], [1, 187, 226, 862, 962]]))

##  Try Simualted Annealing (`sann_iterand`)

In [22]:
@time DiscreteGraphAlgorithms.iterate(
    sann_iterand,
    params_graph_gs;
    log_interval = 100,
)

[ Info: 1 iterations complete with value 0.7742178486118132
[ Info: 101 iterations complete with value 0.7741523144687453
[ Info: 201 iterations complete with value 0.7741690696491843
[ Info: 301 iterations complete with value 0.7741626437929198
[ Info: 401 iterations complete with value 0.7741055399896377
[ Info: 501 iterations complete with value 0.7741421385978549
[ Info: 601 iterations complete with value 0.7741519404263658
[ Info: 701 iterations complete with value 0.7741130208372291
[ Info: 801 iterations complete with value 0.774168273610274
[ Info: 901 iterations complete with value 0.7741599008154696
[ Info: 1001 iterations complete with value 0.7741724552122611
[ Info: 1101 iterations complete with value 0.7741527460561064
[ Info: 1201 iterations complete with value 0.7741172312117071
[ Info: 1301 iterations complete with value 0.7741480753217768
[ Info: 1401 iterations complete with value 0.7741255752340206
[ Info: 1501 iterations complete with value 0.7741417022150786
[ Inf

 23.468593 seconds (14.55 M allocations: 4.185 GiB, 3.06% gc time, 0.25% compilation time)


(0.7742178486118132, [259, 409, 586], SimpleGraph{Int64}(2485, [[39, 121, 855, 912, 997], [173, 342, 672, 758, 826], [108, 161, 637, 667, 876], [49, 319, 369, 487, 591], [209, 286, 517, 593, 929], [302, 432, 487, 638, 713], [151, 188, 596, 683, 773], [183, 359, 543, 665, 982], [72, 608, 746, 853, 884], [134, 320, 418, 861, 955]  …  [50, 104, 136, 332, 461], [349, 367, 591, 759, 945], [218, 370, 510, 894, 977], [281, 379, 434, 487, 721], [15, 172, 533, 717, 804], [80, 330, 528, 549, 898], [178, 198, 341, 366, 811], [67, 123, 191, 588, 730], [58, 337, 634, 773, 983], [1, 187, 226, 862, 962]]))

##  Ant Colony Optimization
- Since ACO is a bit slower for smaller graphs (we're only using 1000 nodes), we can decrease the `max_iter_no_improvement` parameter.
- We'll use this `OptimizationParameters` for both the `genetic_iterand` and `aco_iterand`, so we pass those options in the `opts` keyword

In [17]:
# genetic and aco use similar approaches; we'll give them similar stopping conditions for maximum numbers of iterations
params_graph_pso = OptimizationParameters(
    n_remove,
    graph_wrapped;
    dict_arrays_distance = dict_arrays,
    distance_algorithm = algo,
    max_iter = 2000,
    max_iter_no_improvement = 250,
    opts = Dict{Symbol, Any}(
        :aco_init_colony_with_op_s => true,
        :aco_num_elite => 0.5,
        :aco_population_size => 40,
        :genetic_init_pop_with_op_s => true,
        :genetic_num_elite => 0.5,
        :genetic_population_size => 40,
    ),
    S = S0,
);

@time DiscreteGraphAlgorithms.iterate(
    aco_iterand,
    params_graph_pso;
    log_interval = 50,
)

[ Info: 1 iterations complete with value 0.774237677653346
[ Info: 51 iterations complete with value 0.774237677653346
[ Info: 101 iterations complete with value 0.774237677653346
[ Info: 151 iterations complete with value 0.774237677653346
[ Info: 201 iterations complete with value 0.774237677653346
[ Info: 251 iterations complete with value 0.774237677653346
[ Info: Ant Colony algorithm complete in 251 iterations.


 93.021693 seconds (75.32 M allocations: 20.353 GiB, 1.34% gc time, 0.01% compilation time)


(0.774237677653346, [239, 409, 586], SimpleGraph{Int64}(2485, [[39, 121, 855, 912, 997], [173, 342, 672, 758, 826], [108, 161, 637, 667, 876], [49, 319, 369, 487, 591], [209, 286, 517, 593, 929], [302, 432, 487, 638, 713], [151, 188, 596, 683, 773], [183, 359, 543, 665, 982], [72, 608, 746, 853, 884], [134, 320, 418, 861, 955]  …  [50, 104, 136, 332, 461], [349, 367, 591, 759, 945], [218, 370, 510, 894, 977], [281, 379, 434, 487, 721], [15, 172, 533, 717, 804], [80, 330, 528, 549, 898], [178, 198, 341, 366, 811], [67, 123, 191, 588, 730], [58, 337, 634, 773, 983], [1, 187, 226, 862, 962]]))

In [18]:
@time DiscreteGraphAlgorithms.iterate(
    genetic_iterand,
    params_graph_pso;
    log_interval = 50,
)

[ Info: 1 iterations complete with value 0.774237677653346
[ Info: 51 iterations complete with value 0.774237677653346
[ Info: 101 iterations complete with value 0.774237677653346
[ Info: 151 iterations complete with value 0.774237677653346
[ Info: 201 iterations complete with value 0.774237677653346
[ Info: 251 iterations complete with value 0.774237677653346
[ Info: Genetic algorithm complete in 251 iterations.


 46.507611 seconds (37.94 M allocations: 10.464 GiB, 1.66% gc time, 0.95% compilation time)


(0.774237677653346, [239, 409, 586], SimpleGraph{Int64}(2485, [[39, 121, 855, 912, 997], [173, 342, 672, 758, 826], [108, 161, 637, 667, 876], [49, 319, 369, 487, 591], [209, 286, 517, 593, 929], [302, 432, 487, 638, 713], [151, 188, 596, 683, 773], [183, 359, 543, 665, 982], [72, 608, 746, 853, 884], [134, 320, 418, 861, 955]  …  [50, 104, 136, 332, 461], [349, 367, 591, 759, 945], [218, 370, 510, 894, 977], [281, 379, 434, 487, 721], [15, 172, 533, 717, 804], [80, 330, 528, 549, 898], [178, 198, 341, 366, 811], [67, 123, 191, 588, 730], [58, 337, 634, 773, 983], [1, 187, 226, 862, 962]]))

# Try the wheel graph
- The wheel graph should return 1 in the set of vertices (the central vertex) and any other vertices.

In [39]:
n_remove = 3
algo = :dijkstra_kary
dict_arrays_wheel = spawn_arrays(graph_wheel, algo; )
graph_wheel_wrapped = graph_to_graph_wrapper(graph_wheel);
S0_wheel = get_default_kpp_nodes(graph_wheel, n_remove); # use centrality (some measure, see ?get_default_kpp_nodes) as default starting point

# first, we setup some parameters
params_graph_wheel = OptimizationParameters(
    n_remove,
    graph_wheel_wrapped;
    dict_arrays_distance = dict_arrays_wheel,
    distance_algorithm = algo,
    max_iter = 2000,
    max_iter_no_improvement = 100,
    opts = Dict{Symbol, Any}(
        :aco_population_size => 0.005,
        :aco_num_elite => 0.5,
    ),
    S = S0_wheel,
);

In [41]:
@time DiscreteGraphAlgorithms.iterate(
    gs_iterand,
    params_graph_wheel;
    log_interval = 100,
)

  0.764846 seconds (767.84 k allocations: 173.053 MiB, 35.86% gc time)


[ Info: 1 iterations complete with value 0.9869829962740134
[ Info: 101 iterations complete with value 0.9883737697337104
[ Info: Stochastic gradiest descent complete in 105 iterations.


(0.9883737697337104, [1, 42, 546], SimpleGraph{Int64}(995, [[2, 42], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [7, 9], [8, 10], [9, 11]  …  [987, 989], [988, 990], [989, 991], [990, 992], [991, 993], [992, 994], [993, 995], [994, 996], [995, 997], [546, 996]]))

In [42]:
@time DiscreteGraphAlgorithms.iterate(
    aco_iterand,
    params_graph_wheel;
    log_interval = 100,
)

  1.985087 seconds (4.50 M allocations: 774.549 MiB, 7.42% gc time, 0.31% compilation time)


[ Info: 1 iterations complete with value 0.49799700305736483
[ Info: 101 iterations complete with value 0.9883737293296739
[ Info: Ant Colony algorithm complete in 164 iterations.


(0.9883737293296739, [1, 79, 573], SimpleGraph{Int64}(995, [[2, 79], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [7, 9], [8, 10], [9, 11]  …  [987, 989], [988, 990], [989, 991], [990, 992], [991, 993], [992, 994], [993, 995], [994, 996], [995, 997], [573, 996]]))

# We can verify performance using the Krebs et al. Terrorist Network graph
- Use `read_egl` to read the graph
    - note: if you have a header, use `skip_rows = 1`
    - assumes first column is `i`, second row is `j`, and--if a third row is included--weights
    - see `?read_egl` for more

In [6]:
?read_egl

search: read_egl



Read an edgelist from a .egl file. Returns a GraphWrapper object.

## Constructs

```
read_egl(
    fp::String;
    delim::String = " ",
    edge_weight_default::Float64 = 1.0,
    force_undirected::Bool = false,
    infer_weights::Bool = true,
    skip_rows::Int64 = 0,
)::Union{Nothing, GraphWrapper}
```

## Function Arguments

  * `fp`: file path to edgelist file. Can be .egl, .csv, or other.

## Keyword Arguments

  * `delim`: delimiter in edgelist to use to split rows into columns. Infers if   nothing:

      * if the extension in `fp` is .csv, infers delim as ","
      * if the extension in `fp` is .egl, infers delim as " "
  * `edge_weight_default`: default edge weight value to specify if an   invalid edge weight is found
  * `force_undirected`: force the graph adjacency to be read in as undirected? If   true, any edge from i -> j will also be included as j -> i.
  * `infer_weights`: bool denoting whether or not to infer weighting. 

      * If `true`, looks for inputs with 3 columns, assumed to be 

        i, j, w

        where `i` is the row, `j` is the columns, and `w` is the    specified weight. If only vectors of length 2 are specified,    assumes the graph is unweighted.
      * If `false`, only looks for edges, ignoring potential weight    specification.
  * `skip_rows`: number of lines to skip in input edge weights.    NOTE: use this argument if your file has a header (e.g., skip_rows = 1).


In [7]:
# fine the krebs example graph 
path = get_ref_path("krebs.egl"; check_exist = true)
graph_wrapper_krebs = read_egl(path);

In [18]:
n_remove = 2
algo = :dijkstra_kary

# note: 
dict_arrays_krebs = spawn_arrays(graph_wrapper_krebs.graph, algo; ) 

# first, we setup some parameters
params_graph_krebs = OptimizationParameters(
    n_remove,
    graph_wrapper_krebs;
    dict_arrays_distance = dict_arrays_krebs,
    distance_algorithm = algo,
    max_iter = 1000,
    max_iter_no_improvement = 25,
    opts = Dict{Symbol, Any}(
        :aco_population_size => 5,
        :aco_num_elite => 0.5,
        :genetic_population_size => 5,
        :genetic_num_elite => 0.5,
    ),
    parallel_approach = :serial,
);

In [20]:
@time DiscreteGraphAlgorithms.iterate(
    aco_iterand,
    params_graph_krebs;
    log_interval = 20,
)

[ Info: 1 iterations complete with value 0.6656268496099005
[ Info: 21 iterations complete with value 0.6971031746031745
[ Info: 41 iterations complete with value 0.7729943502824859
[ Info: Ant Colony algorithm complete in 58 iterations.


  0.236151 seconds (341.72 k allocations: 32.891 MiB, 74.76% gc time)


(0.7729943502824859, [4, 18], SimpleGraph{Int64}(121, [[2], [1, 3, 4, 13, 18, 24, 35, 46, 57, 60], [2, 7, 60], [2, 5, 6, 18, 46, 57], [4, 6, 8, 9, 10], [4, 5], [3, 8], [5, 7, 10, 16, 17], [5, 10, 11], [5, 8, 9, 11, 12, 14, 15, 16]  …  [40, 44, 50], [39, 40, 43, 53], [39, 40, 43, 52, 56, 58], [39, 49], [39, 49], [40, 53, 58], [2, 4], [40, 53, 56], [40], [2, 3, 46]]))

In [21]:
@time DiscreteGraphAlgorithms.iterate(
    gs_iterand,
    params_graph_krebs;
    log_interval = 20,
)

[ Info: 1 iterations complete with value 0.7729943502824859
[ Info: 21 iterations complete with value 0.7729943502824859
[ Info: Stochastic gradiest descent complete in 26 iterations.


  0.161333 seconds (29.95 k allocations: 3.081 MiB, 95.61% gc time)


(0.7729943502824859, [4, 18], SimpleGraph{Int64}(121, [[2], [1, 3, 4, 13, 18, 24, 35, 46, 57, 60], [2, 7, 60], [2, 5, 6, 18, 46, 57], [4, 6, 8, 9, 10], [4, 5], [3, 8], [5, 7, 10, 16, 17], [5, 10, 11], [5, 8, 9, 11, 12, 14, 15, 16]  …  [40, 44, 50], [39, 40, 43, 53], [39, 40, 43, 52, 56, 58], [39, 49], [39, 49], [40, 53, 58], [2, 4], [40, 53, 56], [40], [2, 3, 46]]))

In [22]:
@time DiscreteGraphAlgorithms.iterate(
    sann_iterand,
    params_graph_krebs;
    log_interval = 20,
)

[ Info: 1 iterations complete with value 0.6070150659133711
[ Info: 21 iterations complete with value 0.5998116760828625
[ Info: 41 iterations complete with value 0.5941996233521658
[ Info: 61 iterations complete with value 0.5999529190207158
[ Info: 81 iterations complete with value 0.6286064030131826
[ Info: 101 iterations complete with value 0.6570123755716977
[ Info: 121 iterations complete with value 0.5954331450094162
[ Info: 141 iterations complete with value 0.5985875706214692
[ Info: 161 iterations complete with value 0.6021657250470809
[ Info: 181 iterations complete with value 0.5958757062146893
[ Info: 201 iterations complete with value 0.5983992467043314
[ Info: 221 iterations complete with value 0.611939736346516
[ Info: 241 iterations complete with value 0.6264406779661018
[ Info: 261 iterations complete with value 0.5970809792843692
[ Info: 281 iterations complete with value 0.5978625235404897
[ Info: 301 iterations complete with value 0.5955273069679848
[ Info: 321 ite

  0.347297 seconds (1.13 M allocations: 117.842 MiB, 46.63% gc time)


(0.7729943502824859, [4, 18], SimpleGraph{Int64}(121, [[2], [1, 3, 4, 13, 18, 24, 35, 46, 57, 60], [2, 7, 60], [2, 5, 6, 18, 46, 57], [4, 6, 8, 9, 10], [4, 5], [3, 8], [5, 7, 10, 16, 17], [5, 10, 11], [5, 8, 9, 11, 12, 14, 15, 16]  …  [40, 44, 50], [39, 40, 43, 53], [39, 40, 43, 52, 56, 58], [39, 49], [39, 49], [40, 53, 58], [2, 4], [40, 53, 56], [40], [2, 3, 46]]))